##### Copyright 2021 The IREE Authors

In [1]:
#@title Licensed under the Apache License v2.0 with LLVM Exceptions.
# See https://llvm.org/LICENSE.txt for license information.
# SPDX-License-Identifier: Apache-2.0 WITH LLVM-exception

# Dynamic Shapes

This notebook

1. Creates a TensorFlow program with dynamic shapes
2. Imports that program into IREE's compiler
3. Compiles the imported program to an IREE VM bytecode module
4. Tests running the compiled VM module using IREE's runtime
5. Downloads compilation artifacts for use with the native (C API) sample application

In [2]:
#@title General setup

import os
import tempfile

ARTIFACTS_DIR = os.path.join(tempfile.gettempdir(), "iree", "colab_artifacts")
os.makedirs(ARTIFACTS_DIR, exist_ok=True)
print(f"Using artifacts directory '{ARTIFACTS_DIR}'")

Using artifacts directory '/tmp/iree/colab_artifacts'


## Create a program using TensorFlow and import it into IREE

NOTE: as in other domains, providing more information to a compiler allows it
to generate more efficient code. As a general rule, the slowest varying
dimensions of program data like batch index or timestep are safer to treat as
dynamic than faster varying dimensions like image x/y/channel. See
[this paper](https://arxiv.org/pdf/2006.03031.pdf) for a discussion of the
challenges imposed by dynamic shapes and one project's approach to addressing
them.

In [3]:
#@title Define a sample TensorFlow module using dynamic shapes

import tensorflow as tf

# Print version information for future notebook users to reference.
print("TensorFlow version: ", tf.__version__)

class DynamicShapesModule(tf.Module):
  # reduce_sum_1d (dynamic input size, static output size)
  #   e.g. [1, 2, 3] -> 6
  @tf.function(input_signature=[tf.TensorSpec([None], tf.int32)])
  def reduce_sum_1d(self, values):
    return tf.math.reduce_sum(values)
    
  # reduce_sum_2d (partially dynamic input size, static output size)
  #   e.g. [[1, 2, 3], [10, 20, 30]] -> [11, 22, 33]
  @tf.function(input_signature=[tf.TensorSpec([None, 3], tf.int32)])
  def reduce_sum_2d(self, values):
    return tf.math.reduce_sum(values, 0)

  # add_one (dynamic input size, dynamic output size)
  #   e.g. [1, 2, 3] -> [2, 3, 4]
  @tf.function(input_signature=[tf.TensorSpec([None], tf.int32)])
  def add_one(self, values):
    return tf.math.add(values, tf.constant(1, dtype=tf.int32))

TensorFlow version:  2.12.0


In [4]:
%%capture
!python -m pip install iree-compiler iree-tools-tf -f https://openxla.github.io/iree/pip-release-links.html

In [5]:
#@title Import the TensorFlow program into IREE as MLIR

from IPython.display import clear_output

from iree.compiler import tf as tfc

compiler_module = tfc.compile_module(
    DynamicShapesModule(), import_only=True, 
    output_mlir_debuginfo=False)
clear_output()  # Skip over TensorFlow's output.

# Print the imported MLIR to see how the compiler views this program.
print("Dynamic Shapes MLIR:\n```\n%s```\n" % compiler_module)

# Save the imported MLIR to disk.
imported_mlir_path = os.path.join(ARTIFACTS_DIR, "dynamic_shapes.mlir")
with open(imported_mlir_path, "wt") as output_file:
  output_file.write(compiler_module)
print(f"Wrote MLIR to path '{imported_mlir_path}'")

Dynamic Shapes MLIR:
```
module {
  func.func @add_one(%arg0: tensor<?xi32>) -> tensor<?xi32> {
    %0 = stablehlo.constant dense<1> : tensor<i32>
    %1 = shape.shape_of %arg0 : tensor<?xi32> -> tensor<1xindex>
    %2 = stablehlo.dynamic_broadcast_in_dim %0, %1, dims = [] : (tensor<i32>, tensor<1xindex>) -> tensor<?xi32>
    %3 = stablehlo.add %arg0, %2 : tensor<?xi32>
    return %3 : tensor<?xi32>
  }
  func.func @reduce_sum_1d(%arg0: tensor<?xi32>) -> tensor<i32> {
    %0 = stablehlo.constant dense<0> : tensor<i32>
    %1 = stablehlo.reduce(%arg0 init: %0) across dimensions = [0] : (tensor<?xi32>, tensor<i32>) -> tensor<i32>
     reducer(%arg1: tensor<i32>, %arg2: tensor<i32>)  {
      %2 = stablehlo.add %arg1, %arg2 : tensor<i32>
      stablehlo.return %2 : tensor<i32>
    }
    return %1 : tensor<i32>
  }
  func.func @reduce_sum_2d(%arg0: tensor<?x3xi32>) -> tensor<3xi32> {
    %0 = stablehlo.constant dense<0> : tensor<i32>
    %1 = stablehlo.reduce(%arg0 init: %0) across dimensio

## Test the imported program

_Note: you can stop after each step and use intermediate outputs with other tools outside of Colab._

_See the [README](https://github.com/openxla/iree/tree/main/samples/dynamic_shapes#instructions) for more details and example command line instructions._

* _The "imported MLIR" can be used by IREE's generic compiler tools_
* _The "flatbuffer blob" can be saved and used by runtime applications_

_The specific point at which you switch from Python to native tools will depend on your project._

In [6]:
%%capture
!python -m pip install iree-compiler -f https://openxla.github.io/iree/pip-release-links.html

In [7]:
#@title Compile the imported MLIR further into an IREE VM bytecode module

from iree.compiler import compile_str

# Note: we'll use the LLVM CPU backend since it has the best support
# for dynamic shapes among our compiler targets.

flatbuffer_blob = compile_str(compiler_module, target_backends=["llvm-cpu"], input_type="stablehlo")

# Save the compiled program to disk.
flatbuffer_path = os.path.join(ARTIFACTS_DIR, "dynamic_shapes_cpu.vmfb")
with open(flatbuffer_path, "wb") as output_file:
  output_file.write(flatbuffer_blob)
print(f"Wrote compiled program to path '{flatbuffer_path}'")

Wrote compiled program to path '/tmp/iree/colab_artifacts/dynamic_shapes_cpu.vmfb'


In [8]:
%%capture
!python -m pip install iree-runtime -f https://openxla.github.io/iree/pip-release-links.html

In [9]:
#@title Test running the compiled VM module using IREE's runtime

from iree import runtime as ireert

config = ireert.Config("local-task")
ctx = ireert.SystemContext(config=config)
vm_module = ireert.VmModule.from_flatbuffer(ctx.instance, flatbuffer_blob)
ctx.add_vm_module(vm_module)

In [10]:
import numpy as np

# Our @tf.functions are accessible by name on the module named 'module'
dynamic_shapes_program = ctx.modules.module

print(dynamic_shapes_program.reduce_sum_1d(np.array([1, 10, 100], dtype=np.int32)).to_host())
print(dynamic_shapes_program.reduce_sum_2d(np.array([[1, 2, 3], [10, 20, 30]], dtype=np.int32)).to_host())
print(dynamic_shapes_program.reduce_sum_2d(np.array([[1, 2, 3], [10, 20, 30], [100, 200, 300]], dtype=np.int32)).to_host())
print(dynamic_shapes_program.add_one(np.array([1, 10, 100], dtype=np.int32)).to_host())

111
[11 22 33]
[111 222 333]
[  2  11 101]


## Download compilation artifacts

In [11]:
ARTIFACTS_ZIP = "/tmp/dynamic_shapes_colab_artifacts.zip"

print(f"Zipping '{ARTIFACTS_DIR}' to '{ARTIFACTS_ZIP}' for download...")
!cd {ARTIFACTS_DIR} && zip -r {ARTIFACTS_ZIP} .

# Note: you can also download files using Colab's file explorer
try:
  from google.colab import files
  print("Downloading the artifacts zip file...")
  files.download(ARTIFACTS_ZIP)  
except ImportError:
  print("Missing google_colab Python package, can't download files")

Zipping '/tmp/iree/colab_artifacts' to '/tmp/dynamic_shapes_colab_artifacts.zip' for download...
  adding: dynamic_shapes_cpu.vmfb (deflated 63%)
  adding: dynamic_shapes.mlir (deflated 74%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>